## Hackathon 04: Performance Tuning
## Find Avg taxi-in (Flights) time per Description (Carrier)  per Airline Mfg (Planes)
### Take below query (run via RDD's) and Performance Tune it to see how fast you can get it to run

In [0]:
# First confirm have 3 files. If not notify instructor and he'll Email you the files
display(dbutils.fs.ls("dbfs:/FileStore/tables/header_flights.csv"))
display(dbutils.fs.ls("dbfs:/FileStore/tables/header_carriers.csv"))
display(dbutils.fs.ls("dbfs:/FileStore/tables/header_planes.csv"))

path,name,size
dbfs:/FileStore/tables/header_flights.csv,header_flights.csv,136035443


path,name,size
dbfs:/FileStore/tables/header_carriers.csv,header_carriers.csv,37794


path,name,size
dbfs:/FileStore/tables/header_planes.csv,header_planes.csv,432996


In [0]:
%scala

// First run as RDD to get benchmark of how much time to execute the query
val flight1 = sc.textFile("dbfs:/FileStore/tables/header_flights.csv").map(line => line.split(","))        
val carrier1 = sc.textFile("dbfs:/FileStore/tables/header_carriers.csv").map(line => line.split(","))        
val planes1 = sc.textFile("dbfs:/FileStore/tables/header_planes.csv").map(line => line.split(",")).filter(line => line.length == 9)               

val flight_header = flight1.first()
val carrier_header = carrier1.first()
val planes_header = planes1.first()

val flight2 = flight1.filter(x => x != flight_header) 
val carrier2 = carrier1.filter(x => x != carrier_header) 
val planes2 = planes1.filter(x => x != planes_header) 

val flightC = flight2.keyBy(line => line(5))
val carrierC = carrier2.keyBy(line => line(0))

flightC.first()
carrierC.first()

val join1 = flightC.join(carrierC)        
val join1P = join1.map{case (a,b) => (b._1(7), (a,b._1,b._2))} 

val planesP = planes2.keyBy(line => line(0))        
val finalRdd = join1P.join(planesP) 
finalRdd.first()
      
val neededData = finalRdd.map{case (a,b) => ((b._1._3(1), b._2(2)), b._1._2(10).toInt)}    
val avgTaxiInByAir = neededData.groupByKey().mapValues(list => list.sum.toFloat/list.size)   

avgTaxiInByAir.take(10)

flight1: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[38] at map at command-4100678212116257:2
carrier1: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[41] at map at command-4100678212116257:3
planes1: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[45] at filter at command-4100678212116257:4
flight_header: Array[String] = Array(month, dayofmonth, dayofweek, deptime, arrtime, uniquecarrier, flightnum, tailnum, elapsedtime, airtime, arrdelay, depdelay, origin, dest, distance, taxiin, taxiout, cancelled, cancellationcode, diverted)
carrier_header: Array[String] = Array(Code, Description)
planes_header: Array[String] = Array(tailnum, type, manufacturer, issue_date, model, status, aircraft_type, engine_type, year)
flight2: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[46] at filter at command-4100678212116257:10
carrier2: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[47] at filter at command-4100678212116257:11
planes2: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[48] at filter at command-4100678212116257:12
flightC: org.apache.spark.rdd.RDD[(String, Array[String])] = MapPartitionsRDD[49] at keyBy at command-4100678212116257:14
carrierC: org.apache.spark.rdd.RDD[(String, Array[String])] = MapPartitionsRDD[50] at keyBy at command-4100678212116257:15
join1: org.apache.spark.rdd.RDD[(String, (Array[String], Array[String]))] = MapPartitionsRDD[53] at join at command-4100678212116257:20
join1P: org.apache.spark.rdd.RDD[(String, (String, Array[String], Array[String]))] = MapPartitionsRDD[54] at map at command-4100678212116257:21
planesP: org.apache.spark.rdd.RDD[(String, Array[String])] = MapPartitionsRDD[55] at keyBy at command-4100678212116257:23
finalRdd: org.apache.spark.rdd.RDD[(String, ((String, Array[String], Array[String]), Array[String]))] = MapPartitionsRDD[58] at join at command-4100678212116257:24
neededData: org.apache.spark.rdd.RDD[((String, String), Int)] = MapPartitionsRDD[59] at map at command-4100678212116257:28
avgTaxiByAir: org.apache.spark.rdd.RDD[((String, String), Float)] = MapPartitionsRDD[61] at mapValues at command-4100678212116257:29
res13: Array[((String, String), Float)] = Array(((Delta Air Lines Inc.,MCDONNELL DOUGLAS CORPORATION),8.265625), ((Alaska Airlines Inc.,MCDONNELL DOUGLAS),5.9397826), ((Southwest Airlines Co.,BOMBARDIER INC),5.275789), ((Mesa Airlines Inc.,BOEING),13.430187), ((Northwest Airlines Inc.,BOEING),7.36541), ((Southwest Airlines Co.,AIRBUS),7.5576377), ((American Eagle Airlines Inc.,PAIR MIKE E),-2.0257733), ((American Airlines Inc.,EMBRAER),10.972143), ((Frontier Airlines Inc.,MCDONNELL DOUGLAS AIRCRAFT CO),6.6797595), ((JetBlue Airways,BOEING),11.374466))

In [0]:
flightsDF = spark.read.csv("dbfs:/FileStore/tables/header_flights.csv", header=True, inferSchema=True).repartition(2)
carriersDF = spark.read.csv("dbfs:/FileStore/tables/header_carriers.csv", header=True, inferSchema=True).repartition(2)
planesDF = spark.read.csv("dbfs:/FileStore/tables/header_planes.csv", header=True, inferSchema=True).repartition(2)
## airportsDF = spark.read.csv("dbfs:/FileStore/tables/header_airports.csv", header=True, inferSchema=True).repartition(2)

fDF = flightsDF.select("uniquecarrier", "tailnum", "taxiin")      
cDF = carriersDF.select("code", "description")                      
pDF = planesDF.select("tailnum", "manufacturer")                         

fDF.write.format("parquet").save("dbfs:/FileStore/tables/flight2", mode = "overwrite")
cDF.write.format("parquet").save("dbfs:/FileStore/tables/carrier2/", mode = "overwrite")
pDF.write.format("parquet").save("dbfs:/FileStore/tables/plane2/", mode = "overwrite")


In [0]:
flightsDF = spark.read.format("parquet").load("dbfs:/FileStore/tables/flight2/").coalesce(2)
carriersDF = spark.read.format("parquet").load("dbfs:/FileStore/tables/carrier2/").coalesce(1)
planesDF = spark.read.format("parquet").load("dbfs:/FileStore/tables/plane2/").coalesce(1)
# airportsDF = spark.read.format("parquet").load("dbfs:/FileStore/tables/airport1/")

In [0]:
flightsDF.createOrReplaceTempView("flights")
carriersDF.createOrReplaceTempView("carriers")
planesDF.createOrReplaceTempView("planes")
# airportsDF.coalesce(4).createOrReplaceTempView("airports")

In [0]:
spark.catalog.cacheTable("flights")
spark.catalog.cacheTable("carriers")
spark.catalog.cacheTable("planes")

spark.sql("select * from flights").show()
spark.sql("select * from carriers").show()
spark.sql("select * from planes").show()
spark.sql("set spark.sql.shuffle.partitions=2")

+-------------+-------+------+
uniquecarrier|tailnum|taxiin|
+-------------+-------+------+
 9E| 87909E| 22|
 NW| N554NW| 4|
 AA| N3AVAA| 4|
 AA| N3AVAA| 4|
 AA| N3AVAA| 4|
 B6| N238JB| 13|
 UA| N820UA| 10|
 NW| N354NW| 7|
 NW| N354NW| 7|
 9E| 84329E| 12|
 OH| N709CA| 6|
 OH| N709CA| 6|
 OH| N709CA| 6|
 OH| N709CA| 6|
 XE| N18120| 6|
 XE| N18120| 6|
 XE| N18120| 6|
 XE| N18120| 6|
 XE| N18120| 6|
 XE| N18120| 6|
+-------------+-------+------+
only showing top 20 rows

+----+--------------------+
code| description|
+----+--------------------+
 AEC| Astec Air East Inc.|
 PNM| Panama Aviation|
 VAQ| Vacationair|
 VAS| Valley Air Services|
 0JQ| Vision Airlines|
 TP|Tap-Portuguese Ai...|
 AAA|Associated Aviati...|
 SH|Servicio Aereo De...|
 ZL| Affretair|
 CSA| Casino Airlines|
 BAY|Burlington Aeropl...|
 CAI|Crawford Aviation...|
 CRP| Corporate Air|
 FAQ|Fast Air Carrier ...|
 AAQ|Advance Air Charters|
 AQ| Aloha Airlines Inc.|
 ACA|Alaska Central Ai...|
 EXA|Columbia Pacific ...|
 COM| Commutair|
 OB|Oasis Int'l Airli...|
+----+--------------------+
only showing top 20 rows

+-------+-----------------+
tailnum| manufacturer|
+-------+-----------------+
 N17128| BOEING|
 N963DL| BOMBARDIER INC|
 N11206| AIRBUS INDUSTRIE|
 N184US| BOEING|
 N319NB| AIRBUS INDUSTRIE|
 N4YTAA| null|
 N549US| BEECH|
 N833AS| AIRBUS INDUSTRIE|
 N313AE| BOEING|
 N15710| EMBRAER|
 N13624| EMBRAER|
 N717UW| EMBRAER|
 N676SW| BOEING|
 N5BVAA| BOEING|
 N18101| BOEING|
 N362NB| BOEING|
 N624AA| BOEING|
 N667DN| BOEING|
 N590JB|MCDONNELL DOUGLAS|
 N14920| EMBRAER|
+-------+-----------------+
only showing top 20 rows

Out[20]: DataFrame[key: string, value: string]

In [0]:
spark.sql("select c.description, p.manufacturer,  avg(f.taxiin) from flights f JOIN carriers c on f.uniquecarrier = c.code JOIN planes p on f.tailnum = p.tailnum group by c.description, p.manufacturer order by avg(taxiin) desc").show()

+--------------------+--------------------+------------------+
 description| manufacturer| avg(taxiin)|
+--------------------+--------------------+------------------+
Delta Air Lines Inc.| AEROSPATIALE|10.134453781512605|
American Airlines...| AERO COMMANDER| 9.333333333333334|
Continental Air L...| DOUGLAS| 9.306122448979592|
Delta Air Lines Inc.| AIRBUS INDUSTRIE| 9.088145349883073|
Delta Air Lines Inc.| BEECH| 8.977810650887575|
Delta Air Lines Inc.| EMBRAER| 8.915499322187076|
Delta Air Lines Inc.| AEROSPATIALE/ALENIA| 8.866261398176292|
Delta Air Lines Inc.| BOEING| 8.813003318088064|
American Airlines...| AGUSTA SPA| 8.73076923076923|
Delta Air Lines Inc.| AIRBUS| 8.719496145892085|
Delta Air Lines Inc.| CESSNA| 8.681818181818182|
American Airlines...| FREDERICK CHRIS K| 8.651442307692308|
Northwest Airline...|MCDONNELL DOUGLAS...| 8.595238095238095|
American Airlines...| CESSNA| 8.558541266794625|
Delta Air Lines Inc.| null| 8.519719664696991|
Northwest Airline...| PIPER| 8.478915662650602|
Northwest Airline...| DOUGLAS| 8.457446808510639|
Northwest Airline...| MCDONNELL DOUGLAS| 8.452018278750952|
Northwest Airline...| BOMBARDIER INC| 8.437941285767373|
Delta Air Lines Inc.| CANADAIR| 8.435832274459974|
+--------------------+--------------------+------------------+
only showing top 20 rows